In [3]:
import numpy as np
from math import radians, cos, sin, asin, sqrt
import time

x=["28.631301","28.631299","28.631302","28.631303","28.631392","28.631435","28.631431","28.631429","28.631424","28.631421","28.631415","28.631415","28.631411","28.631407","28.631401","28.631401","28.631387","28.631377","28.631371","28.631428","28.631683","28.631777","28.631928","28.631978","28.63178","28.631981","28.63187","28.631996","28.632005","28.632012","28.632021","28.632025","28.632018","28.632023","28.632038","28.632046","28.632053","28.632055","28.632057","28.632053","28.632057","28.632069"]
y=["77.370925","77.370836","77.370678","77.370520","77.37018","77.370619","77.370682","77.370820","77.370941","77.371126","77.371303","77.371499","77.371717","77.372038","77.372807","77.373277","77.373824","77.374038","77.374287","77.374264","77.374096","77.373981","77.373668","77.373364","77.373027","77.372617","77.372209","77.371751","77.371252","77.370647","77.370287","77.369923","77.369545","77.369234","77.368859","77.368400","77.367689","77.366857","77.366056","77.365154","77.364283","77.363496"]
float_x = np.asarray(x, dtype=float)
float_y = np.asarray(y, dtype=float)

length = len(x)
array_dist = []
array_speed = []
array_acceleration = []
array_fuel = []
fuel = 30
total_fuel = fuel
d=0
speed_kmh=0
earlier_speed_instance=0
distance = 0

#Haversine formula to calculate distance
for d in range(length-1):
    a = float(x[d])
    b = float(y[d])
    c = float(x[d+1])
    d = float(y[d+1])
    lat1 = radians(a)
    lat2 = radians(c)
    lon1 = radians(b)
    lon2 = radians(d)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2

    c = 2 * asin(sqrt(a))

    r = 6371
    dist_km = (c * r)
    dist_meter = dist_km*1000
    #print(dist_km," ",dist_meter)
    speed = dist_meter/5
    #print("Speed =",speed,"m/s")
    speed_kmh = (speed*3.6)
    acceleration = (speed_kmh - earlier_speed_instance)/5
    print("Distance covered = ",round(dist_km,3),"km")
    print("Speed =",int(speed_kmh),"km/h")
    if(acceleration<0):
        print("Deceleration =",abs(round(acceleration,2)),"km/h2")
    elif(acceleration>0):
        print("Acceleration =",abs(round(acceleration,2)),"km/h2")
    if(speed_kmh<15):
         net_fuel_consumption = dist_km/8
    elif(speed_kmh<30 and speed_kmh>15):
         net_fuel_consumption = dist_km/12
    elif(speed_kmh<80 and speed_kmh>30):
         net_fuel_consumption = dist_km/16
    elif(speed_kmh>80 and speed_kmh<140):
         net_fuel_comsumption = dist_km/10
    elif(speed_kmh>140):
         net_fuel_comsumption = dist_km/5
    fuel = fuel - net_fuel_consumption
    round_fuel = round(fuel,2)
    print("Current Fuel = ", round_fuel,"litre")
    print(" ")
    distance=distance+dist_km
    earlier_speed_instance = speed_kmh
    array_dist.append(round(dist_km,3))
    array_speed.append(int(speed_kmh))
    array_acceleration.append(round(acceleration,2))
    array_fuel.append(round_fuel)
    #time.sleep(5)
print("Total distance covered = ",round(distance,2)," km")
fuel_eco = distance/(total_fuel-fuel)
print("Fuel Economy = ",round(fuel_eco,2)," km/l")
    
import paho.mqtt.client as paho

BROKER_NAME = "broker.hivemq.com"
PORT = 1883
TOPIC_PUB = "DAFRoute3"
TOPIC_SUB = "DAFRoute3"
fConnected = False

def on_connect(client, userdata, flags, rc):
    print(rc)

    if rc == 0:
        print("Client Connected to Broker!")
        global fConnected
        fConnected = True


def on_message(client, userdata, msg):
    print(msg.topic)
    print(msg.payload.decode("utf-8"))


def on_publish(client, userdata, msg):
    print("")


client = paho.Client("MQTT")
client.on_connect = on_connect
client.on_message = on_message
client.on_publish = on_publish
client.connect(BROKER_NAME,port = PORT)
client.loop_start()
while(fConnected != True):
    time.sleep(1)

while(True):
    time.sleep(1)
    a=0
    while a<length:
        keyInputx = float_x[a]
        keyInputy = float_y[a]
        distP = array_dist[a-1]
        speedP = array_speed[a-1]
        accP = array_acceleration[a-1]
        fuelP = array_fuel[a-1]
        client.publish(TOPIC_PUB, keyInputx)
        client.publish(TOPIC_PUB, keyInputy)
        client.publish(TOPIC_PUB, "Distance Covered =")
        client.publish(TOPIC_PUB, distP)
        client.publish(TOPIC_PUB, "Speed =")
        client.publish(TOPIC_PUB, speedP)
        client.publish(TOPIC_PUB, "Acceleration =")
        client.publish(TOPIC_PUB, accP)
        client.publish(TOPIC_PUB, "Rem. Fuel =")
        client.publish(TOPIC_PUB, fuelP)
        client.publish(TOPIC_PUB," ")
        time.sleep(5)
        a=a+1
    client.publish(TOPIC_PUB,fuel_eco)
    client.loop_stop()
    break
    
print("Published")



Distance covered =  0.009 km
Speed = 6 km/h
Acceleration = 1.25 km/h2
Current Fuel =  30.0 litre
 
Distance covered =  0.015 km
Speed = 11 km/h
Acceleration = 0.97 km/h2
Current Fuel =  30.0 litre
 
Distance covered =  0.015 km
Speed = 11 km/h
Deceleration = 0.0 km/h2
Current Fuel =  30.0 litre
 
Distance covered =  0.035 km
Speed = 24 km/h
Acceleration = 2.77 km/h2
Current Fuel =  29.99 litre
 
Distance covered =  0.043 km
Speed = 31 km/h
Acceleration = 1.22 km/h2
Current Fuel =  29.99 litre
 
Distance covered =  0.006 km
Speed = 4 km/h
Deceleration = 5.32 km/h2
Current Fuel =  29.99 litre
 
Distance covered =  0.013 km
Speed = 9 km/h
Acceleration = 1.05 km/h2
Current Fuel =  29.99 litre
 
Distance covered =  0.012 km
Speed = 8 km/h
Deceleration = 0.24 km/h2
Current Fuel =  29.99 litre
 
Distance covered =  0.018 km
Speed = 13 km/h
Acceleration = 0.9 km/h2
Current Fuel =  29.98 litre
 
Distance covered =  0.017 km
Speed = 12 km/h
Deceleration = 0.11 km/h2
Current Fuel =  29.98 litre
 